In [8]:
!pip install -U langchain
!pip install -U "langchain[openai]"
# Requires Python 3.10+

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.5
    Uninstalling langchain-core-1.0.5:
      Successfully uninstalled langchain-core-1.0.5
  Attempting uninstall: langchain━━━━━━━━━━━━━━━ 0/2 [langchain-core]
    Found existing installation: langchain 1.0.7 0/2 [langchain-core]
    Uninstalling langchain-1.0.7:━━━━━━━━━━━ 0/2 [langchain-core]
      Successfully uninstalled langchain-1.0.70m 0/2 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain]/2 [langchain]


In [11]:
import ast
import json

input_file = "/home/hoangnam/fusion/data/meta_baby.json"     # file bạn chứa các dict
output_file = "products.json"       # file JSON hoàn chỉnh muốn tạo

products = []

with open(input_file, "r", encoding="utf-8") as f:
    # Nếu mỗi product là 1 dòng => duyệt từng dòng
    for line in f:
        line = line.strip()
        if not line:
            continue  # bỏ qua dòng trống
        # Chuyển từ chuỗi dict kiểu Python sang object Python
        obj = ast.literal_eval(line)
        products.append(obj)

# Ghi ra file JSON chuẩn: là 1 mảng các object
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(products, f, ensure_ascii=False, indent=2)

print("Đã tạo file", output_file)


Đã tạo file products.json


In [9]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

model = init_chat_model("gpt-4.1", api_key=OPENAI_API_KEY)



In [12]:
import json

# -------------------------------
# 1. Load model file (VLM outputs)
# -------------------------------
with open("amazon_baby_model_qwen_type_sample_descriptions.json", "r") as f:
    model_data = json.load(f)

# -------------------------------
# 2. Load products.json (CHUẨN JSON)
# -------------------------------
with open("/home/hoangnam/fusion/products.json", "r") as f:
    products = json.load(f)

# -------------------------------
# 3. Map asin → original_description
# -------------------------------
product_map = {
    item["asin"]: item.get("description", "")
    for item in products
}

# -------------------------------
# 4. Merge vào model_data
# -------------------------------
merged = []
missing = []

for item in model_data:
    asin = item.get("asin")
    if asin in product_map:
        item["original_description"] = product_map[asin]
        merged.append(item)
    else:
        missing.append(asin)

# -------------------------------
# 5. Save file merge (optional)
# -------------------------------
with open("merged_products.json", "w") as f:
    json.dump(merged, f, indent=2)

print("Merged items:", len(merged))
print("Missing ASINs:", missing)


Merged items: 7036
Missing ASINs: []


In [18]:
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# ---------------------------
# LOAD 5 ITEM ĐẦU TIÊN
# ---------------------------
with open("merged_products.json", "r") as f:
    data = json.load(f)

items = data[:5]   # lấy 5 phần tử đầu


# ---------------------------
# SETUP LLM + PARSER
# ---------------------------
llm = ChatOpenAI(model="gpt-4.1", temperature=0)
parser = StrOutputParser()

# ---------------------------
# PHASE 1: ANALYSIS
# ---------------------------
analysis_prompt = PromptTemplate(
    input_variables=["title", "img_title_desc", "original_desc"],
    template="""
You are an information extraction assistant.

Product title:
{title}

Descriptions:
1. Image-title-based description: {img_title_desc}
2. Original Amazon description: {original_desc}

Your tasks:
1. Identify all common correct details appearing in both descriptions or implied by the title.
2. Identify useful unique details from each description.
3. Identify contradictions between them.

Return JSON EXACTLY:

{{
  "common_details": [...],
  "unique_details": {{
    "img_title_desc": [...],
    "original_desc": [...]
  }},
  "contradictions": [...]
}}
"""
)
analysis_chain = analysis_prompt | llm | parser


# ------------------ Agent 2: REWRITE (có feedback) ------------------
rewrite_prompt = PromptTemplate(
    input_variables=["title", "analysis", "feedback"],
    template="""
You are a product description rewriting assistant.

Product title:
{title}

Here is a JSON analysis of this product based on the descriptions:
{analysis}

Here is feedback from a fact-checking step about the previous draft (if any):
{feedback}

Your job:
- Write or revise ONE unified product description (2–3 sentences) that:
  - is clearly about the product in the title,
  - keeps all correct and important details from the analysis,
  - fixes any issues mentioned in the feedback (remove or rephrase unsupported details),
  - does NOT invent new facts.

Requirements:
- Use the exact product name from the title at the beginning of the description.
- Make the text sound natural and helpful for an online product page.
- If the feedback says some detail is not supported, do NOT include that detail again.

Return ONLY the fused description as plain text.
"""
)
rewrite_chain = rewrite_prompt | llm | parser


# ---------------------------
# PHASE 3: REFLECTION
# ---------------------------
reflection_prompt = PromptTemplate(
    input_variables=["title", "draft", "img_title_desc", "original_desc"],
    template="""
You are a fact-checking assistant.

Product title:
{title}

Draft fused description:
{draft}

Original descriptions:
1. {img_title_desc}
2. {original_desc}

Your tasks:
1. Check if the draft contains ANY detail not supported by:
   - the product title, OR
   - the original Amazon description, OR
   - the image-title-based description.
2. If all details are supported → return "OK".
3. If any detail is unsupported → return "FAIL" and list which details are wrong.

Return EXACTLY one of these formats:

OK

or

FAIL: [list of unsupported details]
"""
)
reflection_chain = reflection_prompt | llm | parser




In [17]:
def run_loop(item, max_loops=3):
    title = item["title"]
    img_title_desc = item.get("image_title_based_desc", "")
    original_desc = item["original_description"]

    print("\n==============================")
    print(f"RUNNING ITEM: {title}")
    print("==============================")

    # Phase 1: phân tích 1 lần
    analysis = analysis_chain.invoke({
        "title": title,
        "img_title_desc": img_title_desc,
        "original_desc": original_desc
    })
    print("\n--- PHASE 1 ANALYSIS ---")
    print(analysis)

    # feedback ban đầu (chưa có gì)
    feedback = "No feedback yet. This is the first draft."

    last_draft = None

    # Loop Phase 2 ↔ Phase 3
    for i in range(max_loops):
        print(f"\n=== LOOP TURN {i+1} ===")

        # Phase 2: REWRITE với feedback hiện tại
        draft = rewrite_chain.invoke({
            "title": title,
            "analysis": analysis,
            "feedback": feedback,
        })
        last_draft = draft

        print("\n--- PHASE 2 DRAFT ---")
        print(draft)

        # Phase 3: CHECK
        reflection = reflection_chain.invoke({
            "title": title,
            "draft": draft,
            "img_title_desc": img_title_desc,
            "original_desc": original_desc
        })

        print("\n--- PHASE 3 CHECK ---")
        print(reflection)

        # Nếu OK thì dừng
        if reflection.strip().startswith("OK"):
            print("\n✅ FINAL OUTPUT:")
            print(draft)
            return draft

        # Nếu FAIL → dùng nguyên reflection làm feedback cho vòng sau
        feedback = reflection

    # Hết max_loops mà vẫn FAIL → trả bản cuối cùng
    print("\n❌ Failed after max loops — returning last draft")
    return last_draft


In [19]:
# ---------------------------
# RUN FOR 5 ITEMS
# ---------------------------
final_outputs = []

for item in items:
    out = run_loop(item)
    final_outputs.append({
        "asin": item["asin"],
        "final_description": out
    })

print("\n\n==========================")
print("FINISHED ALL 5 ITEMS")
print("==========================")
print(json.dumps(final_outputs, indent=2))


RUNNING ITEM: Arm's Reach Original Co-sleeper 100% Cotton White Sheet

--- PHASE 1 ANALYSIS ---
{
  "common_details": [
    "Arm's Reach Original Co-sleeper",
    "Sheet",
    "White color",
    "Cotton material",
    "Designed for babies"
  ],
  "unique_details": {
    "img_title_desc": [
      "100% cotton",
      "Soft and comfortable bedding option",
      "Provides a cozy sleeping environment",
      "High-quality cotton",
      "Durability",
      "Breathability",
      "Ensures a safe and healthy sleep experience for infants",
      "Clean and neutral aesthetic",
      "Versatile choice for various nursery themes"
    ],
    "original_desc": [
      "Cotton blend",
      "Fitted sheet",
      "Specifically designed for Arm's Reach co-sleepers"
    ]
  },
  "contradictions": [
    "img_title_desc claims the sheet is 100% cotton, while original_desc states it is a cotton blend."
  ]
}

=== LOOP TURN 1 ===

--- PHASE 2 DRAFT ---
Arm's Reach Original Co-sleeper 100% Cotton White Sh

Đánh giá từng trường hợp
(1) Arm's Reach Original Co-sleeper Sheet

Final:

“...soft, high-quality sheet ... Made from 100% cotton...”

Điểm tốt:

Bỏ được chữ “fitted” (vì Phase 3 chê).

Giữ được: soft, cozy, white, cho co-sleeper → đều có trong img_title_desc + original.

Điểm hơi risk:

100% cotton vs original nói cotton blend → mình đang tin title + img_title_desc hơn là original description.

high-quality là adj thêm vào, không phải fact rõ ràng.

👉 Nếu muốn an toàn dữ liệu hơn: chỉnh Phase 2/3 để ưu tiên original description khi có conflict → lúc đó câu nên nói “cotton blend” hoặc né luôn %.

(2) Blind Cord Wind Ups

Final:

“durable plastic devices… keep cords neatly out of reach… neutral color, modern look…”

Rất ổn:

Hiểu đúng: đây là device quấn dây, không phải “cordless blind”.

Có “durable plastic”, “clip around window blind cords”, “retract excess line” → bám đúng original.

Risk nhẹ:

“modern look” là diễn đạt thêm, nhưng dựa được trên “neutral color + blends with decor” nên cũng chấp nhận được.

(3) Northstate Superyard Playgate

Rất tốt:

Có đủ: 6 panels, 18.5 sqft, extendable to 34.4 sqft, indoor/outdoor, carrying strap.

Không bịa thêm tính năng lạ.

“sleek, decorative design, easy-to-maintain finish” là adj nhẹ, vẫn dựa được vào img_title_desc (“decorative, easy maintenance”).

(4) Playtex Drop-Ins Starter Set

Phase 3 làm việc rất tốt:

Vòng 1 bị chê: “complete feeding solution”, “convenient…”, “ideal for parents…”.

Vòng 2 sửa lại: mô tả chính xác set gồm:

5 chai (2×4oz, 3×8oz)

liner 2 loại

4 kiểu núm ti

1 ống 8oz có Eazy-Feed system.

Final hiện tại bám khá sát original, gần như không hallucination.

(5) Terry Contoured Changing Pad Cover

Ổn:

Lấy được: Terry cloth, elasticized, giữ ấm, dễ giặt, bền, nên mua nhiều cái.

“stylish and functional nursery accessory” là đứng giữa image + original, chấp nhận được.

3. Kết luận nhanh

Về logic loop:

Phase 1: phân tích + bắt contradiction tốt.

Phase 2: viết khá đẹp, giữ được key facts.

Phase 3: bắt lỗi được hầu hết chi tiết nguy hiểm, đặc biệt case Playtex.

Điểm còn cân nhắc:

Case Arm’s Reach: chuyện 100% cotton vs cotton blend là kiểu conflict “không có câu trả lời đúng” → nếu muốn an toàn tuyệt đối, nên:

Ưu tiên original description, hoặc

Né luôn tỉ lệ (“soft cotton sheet” thay vì “100% cotton”).